In [1]:
pip install albumentations

  Obtaining dependency information for albumentations from https://files.pythonhosted.org/packages/9b/f6/c486cedb4f75147232f32ec4c97026714cfef7c7e247a1f0427bc5489f66/albumentations-1.3.1-py3-none-any.whl.metadata
  Obtaining dependency information for opencv-python-headless>=4.1.1 from https://files.pythonhosted.org/packages/e3/10/31b27a7473043eb5317f698ede00e7e129b2de378903bfe0bb4d785a7baf/opencv_python_headless-4.8.1.78-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/125.7 kB ? eta -:--:--
   ---------------------------------------  122.9/125.7 kB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 125.7/125.7 kB 2.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/38.0 MB ? eta -:--:--
   ---------------------------------------- 0.2/38.0 MB 5.3 MB/s eta 0:00:08
    --------------------------------------- 0.6/38.0 MB 7.1 MB/s eta 0:00:06
   - -------------------------------------- 1.1/38.0 MB 8.5 MB/s eta 0:00:05
   - ----

In [ ]:
import pandas as pd
import numpy as np
import os
import json
import cv2
import random
import sklearn
import pydot
import tqdm
from datetime import datetime

from matplotlib import image as mpimg
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.metrics import confusion_matrix, f1_score, classification_report
import yaml
import torch
from pathlib import Path

import ultralytics
from ultralytics import YOLO
from albumentations import (
    HorizontalFlip,
    VerticalFlip,
    RandomRotate90,
    RandomBrightnessContrast,
    RandomGamma,
    Compose,
    ImageOnlyTransform,
    Resize,
)

In [ ]:
# Initialize the YOLO model
model = YOLO('yolov8n.pt')

# Define the paths
data_dir = 'D:/Skripsi/dataset/'
train_images_dir = os.path.join(data_dir, 'images', 'train')
train_labels_dir = os.path.join(data_dir, 'labels', 'train')

# Create directories for augmented data
augmented_images_dir = 'D:/Benji/dataset_aug/images/train'
augmented_labels_dir = 'D:/Benji/dataset_aug/labels/train'

os.makedirs(augmented_images_dir, exist_ok=True)
os.makedirs(augmented_labels_dir, exist_ok=True)

# Augmentation transformations
transform = Compose([
    HorizontalFlip(p=0.5),  # Horizontal flip with a probability of 50%
    VerticalFlip(p=0.5),    # Vertical flip with a probability of 50%
    RandomRotate90(p=0.5),  # Random 90-degree rotations with a probability of 50%
    RandomBrightnessContrast(p=0.2),  # Random brightness and contrast adjustments
    RandomGamma(p=0.2),  # Random gamma adjustments
    ImageOnlyTransform(Resize(416, 416)),  # Resize the image to the YOLO input size
])

# List all image files in the training images directory
image_files = [f for f in os.listdir(train_images_dir) if f.endswith('.jpg')]

# Iterate over each image file and apply augmentation
for image_file in image_files:
    # Load the image
    image_path = os.path.join(train_images_dir, image_file)
    image = cv2.imread(image_path)

    # Load the corresponding label (assuming it's in YOLO format)
    label_file = os.path.splitext(image_file)[0] + '.txt'
    label_path = os.path.join(train_labels_dir, label_file)

    # Apply augmentation to the image
    augmented = transform(image=image)

    # Save the augmented image
    augmented_image = augmented['image']
    augmented_image_path = os.path.join(augmented_images_dir, image_file)
    cv2.imwrite(augmented_image_path, augmented_image)

    # Save the corresponding label (assuming it's in YOLO format)
    augmented_label_path = os.path.join(augmented_labels_dir, label_file)
    with open(label_path, 'r') as label_file:
        augmented_label = label_file.read()
    with open(augmented_label_path, 'w') as label_file:
        label_file.write(augmented_label)